In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# user-model import
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my_playground/models')

In [3]:
from google.colab import files # 데이터 불러오기
file_uploaded=files.upload()   # chap06/data/dogs-vs-cats.zip 데이터 불러오기

Saving dogs-vs-cats.zip to dogs-vs-cats.zip


In [4]:
!unzip dogs-vs-cats.zip -d dogs-vs-cats/  #dogs-vs-cats 폴더 만들어 압축 풀기

Archive:  dogs-vs-cats.zip
   creating: dogs-vs-cats/Cat/
  inflating: dogs-vs-cats/Cat/cat.0.jpg  
  inflating: dogs-vs-cats/Cat/cat.1.jpg  
  inflating: dogs-vs-cats/Cat/cat.10.jpg  
  inflating: dogs-vs-cats/Cat/cat.100.jpg  
  inflating: dogs-vs-cats/Cat/cat.101.jpg  
  inflating: dogs-vs-cats/Cat/cat.102.jpg  
  inflating: dogs-vs-cats/Cat/cat.103.jpg  
  inflating: dogs-vs-cats/Cat/cat.104.jpg  
  inflating: dogs-vs-cats/Cat/cat.105.jpg  
  inflating: dogs-vs-cats/Cat/cat.106.jpg  
  inflating: dogs-vs-cats/Cat/cat.107.jpg  
  inflating: dogs-vs-cats/Cat/cat.108.jpg  
  inflating: dogs-vs-cats/Cat/cat.109.jpg  
  inflating: dogs-vs-cats/Cat/cat.11.jpg  
  inflating: dogs-vs-cats/Cat/cat.110.jpg  
  inflating: dogs-vs-cats/Cat/cat.111.jpg  
  inflating: dogs-vs-cats/Cat/cat.112.jpg  
  inflating: dogs-vs-cats/Cat/cat.113.jpg  
  inflating: dogs-vs-cats/Cat/cat.114.jpg  
  inflating: dogs-vs-cats/Cat/cat.115.jpg  
  inflating: dogs-vs-cats/Cat/cat.116.jpg  
  inflating: dogs-vs-cat

## Library

In [5]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import pandas as pd
import LeNet5

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, mode):
        if mode == "test": #test 도 val과 같이 처리
            mode = "val"
        return self.data_transform[mode](img)

In [8]:
class DogvsCatDataset(Dataset):
    def __init__(self, file_list, transform=None, mode='train'):
        self.file_list = file_list
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.mode)
        label = img_path.split('/')[-1].split('.')[0]
        if self.mode == "test":
            return img_transformed
        else:
            if label == 'dog':
                label = 1
            elif label == 'cat':
                label = 0
            return img_transformed, label

In [9]:
class Trainer:
    def __init__(self, model_name = "LeNet5"):
        self.cat_path = "dogs-vs-cats/Cat/"
        self.dog_path = "dogs-vs-cats/Dog/"
        self.model_name = model_name
        self.save_path = f"./best_{model_name}_model.pth"

        self.size = 224
        self.mean = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

        self.BATCH_SIZE = 32
        self.NUM_WORKERS = 0
        self.EPOCH = 10
        self.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.SEED = 42
        seed_everything(self.SEED)

    def setup(self):
        """ setup """
        cat_images_filepaths = sorted([os.path.join(self.cat_path, f) for f in os.listdir(self.cat_path)])
        dog_images_filepaths = sorted([os.path.join(self.dog_path, f) for f in os.listdir(self.dog_path)])
        images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
        correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]
        random.shuffle(correct_images_filepaths)

        train_images_filepaths = correct_images_filepaths[:400]
        val_images_filepaths = correct_images_filepaths[400:-10]
        test_images_filepaths = correct_images_filepaths[-10:]

        train_dataset = DogvsCatDataset(train_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='train')
        val_dataset = DogvsCatDataset(val_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='val')
        test_dataset = DogvsCatDataset(test_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='test')

        self.train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=self.BATCH_SIZE,
            shuffle=True,
            num_workers=self.NUM_WORKERS,
            pin_memory=True
        )
        self.val_dataloader = DataLoader(
                    dataset=val_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )
        self.test_dataloader = DataLoader(
                    dataset=test_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )

        self.model_dict = {"LeNet5": LeNet5.LeNet_5()} # 점차 쌓아 나감.
        self.model = self.model_dict[self.model_name]
        self.loss_fn = nn.CrossEntropyLoss() #criterion
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)

    def train(self):
        self.model.to(self.DEVICE)
        self.loss_fn.to(self.DEVICE)
        best_acc = 0.0

        for epoch in range(self.EPOCH):
            self.model.train()
            print('Epoch {}/{}'.format(epoch + 1, self.EPOCH))
            print('-'*20)
            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(self.train_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(self.train_dataloader.dataset)
            epoch_acc = epoch_corrects.double() / len(self.train_dataloader.dataset)
            print('[Train] Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)) #train은 필요 없을 것 같다.

            val_loss, val_acc = self.valid()
            print('[Val] Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

            if val_acc > best_acc:
                best_acc = val_acc
                best_model = self.model.state_dict()
                torch.save(self.model.state_dict(), self.save_path)

    def valid(self):
        self.model.eval()
        val_loss, val_corrects = 0.0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(self.val_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

            val_loss = val_loss / len(self.val_dataloader.dataset)
            val_acc = val_corrects.double() / len(self.val_dataloader.dataset)
        return val_loss, val_acc

    def test(self):
        model = LeNet5.LeNet_5().to(self.DEVICE)
        model.load_state_dict(torch.load(self.save_path))
        model.eval()
        id_list, pred_list = [] , []
        _id=0
        with torch.no_grad():
            for inputs in tqdm(self.test_dataloader):
                for img in inputs:
                    img = img.unsqueeze(0) #왜 test 만 unsqueeze? ->  input차원을 맞춰주기 위해서
                    img = img.to(self.DEVICE)

                    outputs = model(img)
                    preds = F.softmax(outputs, dim=1)[:, 1].tolist()
                    id_list.append(_id)
                    pred_list.append(preds[0])

        res = pd.DataFrame({
            'id': id_list,
            'label': pred_list
        })

        res.sort_values(by='id', inplace=True)
        res.reset_index(drop=True, inplace=True)

        res.to_csv('LesNet.csv', index=False)
        return res

In [10]:
trainer = Trainer()

In [11]:
trainer.setup()

In [12]:
trainer.train()

Epoch 1/10
--------------------


<ipython-input-9-8472b2b4d3c5>:70: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6989 Acc: 0.4900


<ipython-input-9-8472b2b4d3c5>:102: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.7004 Acc: 0.5217
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6647 Acc: 0.6100


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.7003 Acc: 0.5435


In [13]:
trainer.test()

<ipython-input-9-8472b2b4d3c5>:124: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

,id,label
0,0,0.416739
1,0,0.563102
2,0,0.503036
3,0,0.530632
4,0,0.555609
5,0,0.559442
6,0,0.439953
7,0,0.510274
8,0,0.514703
9,0,0.492104
